In [ ]:
import pandas
import cv2
import numpy as np
import time
import math
from datetime import timedelta
import matplotlib.pyplot as plt

In [ ]:
# map_data = pandas.read_csv("biomassmini.csv", delimiter=";", header=None)
map_data = pandas.read_csv("full.csv", delimiter=";", header=None)

res = 4032*3040
biomass = map_data[11]
biomass = biomass/res * 100
bio = biomass.astype(np.uint8)

jet_custom = cv2.applyColorMap(np.arange(256, dtype=np.uint8), cv2.COLORMAP_JET)

In [ ]:
# latitude difference calcs
lat = np.asarray(map_data[2])

lat = np.subtract(lat, lat[0])

lat2 = np.empty(lat.shape)
np.copyto(lat2, lat)
lat2 = np.delete(lat2, 0)
lat = np.delete(lat, -1)
dx = np.subtract(lat,lat2)*1000000

In [ ]:
# longitude difference calcs
long = np.asarray(map_data[3])

long = np.subtract(long, long[0])

long2 = np.empty(long.shape)
np.copyto(long2, long)
long2 = np.delete(long2, 0)
long = np.delete(long, -1)
dy = np.subtract(long,long2)*1000000 #(1 = 0.111 m)

In [ ]:
dirs = [ [1,0], [1,1], [0,1], [-1,1], [-1,0], [-1,-1], [0,-1], [1,-1] ]
def getxy(r, theta):
    if theta>337.5 or theta<22.5:
        idx = 0
    elif theta<67.5:
        idx = 1
    elif theta<112.5:
        idx = 2
    elif theta<157.5:
        idx = 3
    elif theta<202.5:
        idx = 4
    elif theta<247.5:
        idx = 5
    elif theta<292.5:
        idx = 6
    else:
        idx = 7
    fct = int(r/25)
    return np.dot(fct,dirs[idx])

In [ ]:
def cart2pol(x, y):
    rho = np.sqrt(x**2 + y**2)
    phi = np.arctan2(y, x)
    ang = math.degrees(phi)
    if (ang<0): ang = 360+ang
    return(rho, ang)

x = []
y = []

for n in range(len(dx)):
    r, t = cart2pol(dx[n], dy[n])
    i, j = getxy(r, t)
    x.append(i)
    y.append(j)
    
# r, theta = cart2pol(3,4)
# print(x, y)

In [ ]:
map_img = np.ones((2048,2048,3), dtype=np.uint8)# * 255

blc = 5
i, j = 100, 300
for n in range(len(x)):
    i, j = i+x[n], j+y[n]
    x1, x2 = blc*i, blc*i+blc
    y1, y2 = blc*j, blc*j+blc
    map_img[x1:x2, y1:y2] = jet_custom[bio[n]]

cv2.imwrite("biomap.png", map_img)
plt.imshow(map_img), plt.axis('off')
